In [6]:
from __future__ import division, print_function
from keras.layers import Convolution2D
from keras.layers import ZeroPadding2D
from keras.layers import MaxPooling2D

In [ ]:
def ConvBlock(model, layers, filters):
    for i in range(0, layers):
        model.add(ZeroPadding2D(1, 1))
        model.add(Convolution2D(filters, 3, 3, activation='relu'))
    model.add(Ma)